# load packages

In [ ]:
import pandas as pd

In [ ]:
import os

In [ ]:
from datetime import datetime

In [ ]:
from zoneinfo import ZoneInfo

In [ ]:
from dateutil.relativedelta import relativedelta

In [ ]:
import matplotlib.pyplot as plt

# load datasets

## demo case

In [ ]:
# This query represents dataset "AD_GWAS_case" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_81777961_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (1568293, 44826537)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 0 
                        AND is_selectable = 1) 
                    AND is_standard = 0 )) criteria ) )"""

case_demo = pd.read_gbq(
    dataset_81777961_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

case_demo.head(5)

## condition case

In [ ]:
# This query represents dataset "AD_GWAS_case" for domain "condition" and was generated for All of Us Controlled Tier Dataset v8
dataset_81777961_condition_sql = """
    SELECT
        c_occurrence.person_id,
        c_occurrence.condition_start_datetime,
        c_occurrence.condition_source_value 
    FROM
        ( SELECT
            * 
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` c_occurrence 
        WHERE
            (
                condition_source_concept_id IN (SELECT
                    DISTINCT c.concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                JOIN
                    (SELECT
                        CAST(cr.id as string) AS id       
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                    WHERE
                        concept_id IN (1568293, 35207356, 35207357, 35207358, 35207359, 44826537)       
                        AND full_text LIKE '%_rank1]%'      ) a 
                        ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                        OR c.path LIKE CONCAT('%.', a.id) 
                        OR c.path LIKE CONCAT(a.id, '.%') 
                        OR c.path = a.id) 
                WHERE
                    is_standard = 0 
                    AND is_selectable = 1)
            )  
            AND (
                c_occurrence.PERSON_ID IN (SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (SELECT
                        person_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
                    WHERE
                        has_whole_genome_variant = 1 ) 
                    AND cb_search_person.person_id IN (SELECT
                        criteria.person_id 
                    FROM
                        (SELECT
                            DISTINCT person_id, entry_date, concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                        WHERE
                            (concept_id IN(SELECT
                                DISTINCT c.concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                            JOIN
                                (SELECT
                                    CAST(cr.id as string) AS id       
                                FROM
                                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                                WHERE
                                    concept_id IN (1568293, 44826537)       
                                    AND full_text LIKE '%_rank1]%'      ) a 
                                    ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                    OR c.path LIKE CONCAT('%.', a.id) 
                                    OR c.path LIKE CONCAT(a.id, '.%') 
                                    OR c.path = a.id) 
                            WHERE
                                is_standard = 0 
                                AND is_selectable = 1) 
                            AND is_standard = 0 )) criteria ) )
            )) c_occurrence"""

case_icd = pd.read_gbq(
    dataset_81777961_condition_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

case_icd.head(5)

## demo control

In [ ]:
# This query represents dataset "AD_GWAS_control_srWGS" for domain "person" and was generated for All of Us Controlled Tier Dataset v8
dataset_90165111_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_whole_genome_variant = 1 ) )"""

control_demo = pd.read_gbq(
    dataset_90165111_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

control_demo.head(5)

## PCA and GIA

In [ ]:
gia = pd.read_csv('AOU.v8.GIA_PCA.txt', sep  ='\t')
gia.head()

# make controls w no cases

In [ ]:
control_no_case = control_demo[~control_demo['person_id'].isin(case_demo['person_id'])]
print(len(control_no_case.index))
control_no_case.head()

# clean gia

In [ ]:
gia_clean = gia[['research_id', 'ancestry_pred']]
gia_clean.rename(columns = {'research_id' : 'person_id'}, inplace = True)
gia_clean.head()

# make cases by rule of 2

## make person id list

In [ ]:
id_list = case_icd['person_id'].unique().tolist()
len(id_list)

## loop through dataframe and only extract people that have at least 2 ICD codes

In [ ]:
two_instances = []
for id in id_list:
    id_df = case_icd[case_icd['person_id'].isin([id])]
    id_df.drop_duplicates(inplace = True)
    id_df.reset_index(inplace = True, drop = True)
    if len(id_df.index) >= 2:
        two_instances.append(id)

## check work and create filtered df

In [ ]:
print(len(two_instances))
two_instances

In [ ]:
case_icd_two = case_icd[case_icd['person_id'].isin(two_instances)]
case_icd_two.head()

In [ ]:
case_icd_one = case_icd[~case_icd['person_id'].isin(two_instances)]
case_icd_one.head()

## get first date

### clean up dataframe

In [ ]:
case_icd_two.drop(columns = ['condition_source_value'], inplace = True)
case_icd_two['condition_start_datetime'] = pd.to_datetime(case_icd_two['condition_start_datetime'])
case_icd_two.sort_values(by = ['person_id', 'condition_start_datetime'], inplace = True)
print(case_icd_two['condition_start_datetime'].dtype)
case_icd_two.head()

### get first date

In [ ]:
case_icd_first = case_icd_two.drop_duplicates(keep = 'first', subset = 'person_id')
print(len(case_icd_first.index))
case_icd_first.head()

# clean demo dfs

## filter to males and females

In [ ]:
case_demo['sex_at_birth'].unique()

In [ ]:
case_demo_clean = case_demo[case_demo['sex_at_birth'].isin(['Male', 'Female'])]
print(case_demo_clean['sex_at_birth'].unique())
case_demo_clean.head()

In [ ]:
control_demo_clean = control_demo[control_demo['sex_at_birth'].isin(['Male', 'Female'])]
print(control_demo_clean['sex_at_birth'].unique())
control_demo_clean.head()

## recode sex column (male = 1, female = 2)

In [ ]:
case_demo_clean['sex_at_birth'] = case_demo_clean['sex_at_birth'].str.replace('Male', '1')
case_demo_clean['sex_at_birth'] = case_demo_clean['sex_at_birth'].str.replace('Female', '2')
print(case_demo_clean['sex_at_birth'].unique())
case_demo_clean.head()

In [ ]:
control_demo_clean['sex_at_birth'] = control_demo_clean['sex_at_birth'].str.replace('Male', '1')
control_demo_clean['sex_at_birth'] = control_demo_clean['sex_at_birth'].str.replace('Female', '2')
print(control_demo_clean['sex_at_birth'].unique())
control_demo_clean.head()

## change DOB date type

In [ ]:
case_demo_clean['date_of_birth'] = pd.to_datetime(case_demo_clean['date_of_birth'])
print(case_demo_clean['date_of_birth'].dtype)
case_demo_clean.head()

In [ ]:
control_demo_clean['date_of_birth'] = pd.to_datetime(control_demo_clean['date_of_birth'])
print(case_demo_clean['date_of_birth'].dtype)
control_demo_clean.head()

## calculate age at last data release for controls

### calculate age

In [ ]:
print(datetime(2025, 10, 1, 0, 0, 0))

In [ ]:
control_demo_clean['AGE'] = (datetime(2025, 10, 1, 0, 0, 0)).replace(tzinfo = ZoneInfo("UTC")) - control_demo_clean['date_of_birth']
control_demo_clean['AGE'] = control_demo_clean['AGE'].astype(str).str.replace(' days','')
control_demo_clean['AGE'] = control_demo_clean['AGE'].astype(float)
control_demo_clean['AGE'] = control_demo_clean['AGE'] / 365.2425
control_demo_clean.head()

### remove DOB column

In [ ]:
control_demo_clean.drop(columns = ['date_of_birth'], inplace = True)
control_demo_clean.head()

## calculate age at first condition in cases

### merge icd & demo dfs

In [ ]:
case_icd_demo = case_icd_first.merge(case_demo_clean, how = 'inner', on = 'person_id')
print(len(case_icd_demo.index))
case_icd_demo.head()

### calculate age

In [ ]:
case_icd_demo['condition_start_datetime'] = case_icd_demo['condition_start_datetime'].dt.date
case_icd_demo['date_of_birth'] = case_icd_demo['date_of_birth'].dt.date
case_icd_demo['AGE'] = case_icd_demo['condition_start_datetime'] - case_icd_demo['date_of_birth']
case_icd_demo['AGE'] = case_icd_demo['AGE'].astype(str).str.replace(' days, 0:00:00','')
case_icd_demo['AGE']  =case_icd_demo['AGE'].astype(float)
case_icd_demo['AGE'] = case_icd_demo['AGE'] / 365.2425
case_icd_demo.head()

### drop dob & condition start columns

In [ ]:
case_icd_demo.drop(columns=['date_of_birth','condition_start_datetime'], inplace = True)
case_icd_demo.head()

# examine age distribution

In [ ]:
case_icd_demo['AGE'].describe()

In [ ]:
control_demo_clean['AGE'].describe()

# filter to min age of 65

In [ ]:
control_min_age = control_demo_clean[control_demo_clean['AGE'] >= 65]
print(len(control_min_age.index))
control_min_age.head()

# add AD column

In [ ]:
case_min_age['AD'] = 1
case_min_age.head()

In [ ]:
control_min_age['AD'] = 0
control_min_age.head()

# concatenate!

In [ ]:
case_control = pd.concat([control_min_age, case_min_age], axis = 0)
print(len(case_control.index))
case_control.head()

# merge w gia

In [ ]:
case_control_gia = case_control.merge(gia_clean, how = 'inner', on = 'person_id')
print(len(case_control_gia.index))
print(case_control_gia[['ancestry_pred', 'AD']].value_counts())
print(case_control_gia['AD'].value_counts())
case_control_gia.head()

# create sample lists

## multi-ancestry

In [ ]:
all_sample = case_control[['person_id']]
all_sample.head()

## EUR

In [ ]:
eur_case_control = case_control_gia[case_control_gia['ancestry_pred'].isin(['eur'])]
print(len(eur_case_control.index))
print(eur_case_control['AD'].value_counts())
eur_case_control.head()

In [ ]:
eur_sample=eur_case_control[['person_id']]
eur_sample.head()

# export sample lists

In [ ]:
all_sample.to_csv('AOU.AD.ALL.sample_list.txt', sep = '\t', header = None, index = None)

In [ ]:
eur_sample.to_csv('AOU.AD.EUR.sample_list.txt', sep = '\t', header = None, index = None)

# read in PCs

## eigenvalues

In [ ]:
eur_eigenval = pd.read_csv('AOU.AD.EUR.PCA_cleaned.eigenval', header = None)
print(len(eur_eigenval.index))
eur_eigenval.head()

In [ ]:
all_eigenval = pd.read_csv('AOU.AD.ALL.PCA_cleaned.eigenval', header = None)
print(len(all_eigenval.index))
all_eigenval.head()

## eigenvectors

In [ ]:
eur_eigenvec = pd.read_csv('AOU.AD.EUR.PCA_cleaned.eigenvec', sep = '\t', header = None)
print(len(eur_eigenvec.index))
eur_eigenvec.head()

In [ ]:
all_eigenvec = pd.read_csv('AOU.AD.ALL.PCA_cleaned.eigenvec', sep = '\t', header = None)
print(len(all_eigenvec.index))
all_eigenvec.head()

# make scree plots

## clean files

In [ ]:
eur_eigenval['PC'] = list(range(1,21))
eur_eigenval.rename(columns = {0 : 'EIGENVAL'}, inplace = True)
eur_eigenval['VARIANCE'] = eur_eigenval['EIGENVAL']/eur_eigenval['EIGENVAL'].sum()
eur_eigenval.head()

In [ ]:
all_eigenval['PC'] = list(range(1, 21))
all_eigenval.rename(columns = {0 : 'EIGENVAL'}, inplace = True)
all_eigenval['VARIANCE'] = all_eigenval['EIGENVAL']/all_eigenval['EIGENVAL'].sum()
all_eigenval.head()

## make plots

In [ ]:
plt.plot(eur_eigenval['PC'], eur_eigenval['VARIANCE'])
plt.xlabel('PC')
plt.ylabel('Variance')
plt.title('EUR AD GWAS Scree Plot')
plt.xticks(list(range(1, 21)))

In [ ]:
plt.plot(all_eigenval['PC'], all_eigenval['VARIANCE'])
plt.xlabel('PC')
plt.ylabel('Variance')
plt.title('ALL AD GWAS Scree Plot')
plt.xticks(list(range(1, 21)))

# subset eigenvec files

In [ ]:
eur_eigenvec_sub = eur_eigenvec[[0, 1, 2, 3]]
eur_eigenvec_sub.rename(columns={0 : 'person_id',
                                 1 : 'PC1',
                                 2 : 'PC2',
                                 3 : 'PC3'}, inplace = True)
eur_eigenvec_sub.head()

In [ ]:
all_eigenvec_sub = all_eigenvec[[0, 1, 2, 3, 4]]
all_eigenvec_sub.rename(columns={0 : 'person_id',
                                 1 : 'PC1',
                                 2 : 'PC2',
                                 3 : 'PC3',
                                 4 : 'PC4'}, inplace = True)
all_eigenvec_sub.head()

# merge with pheno files

In [ ]:
eur_pheno_pc=eur_case_control.merge(eur_eigenvec_sub, on = 'person_id')
eur_pheno_pc.drop(columns = ['ancestry_pred'], inplace = True)
print(len(eur_pheno_pc.index))
print(eur_pheno_pc['AD'].value_counts())
eur_pheno_pc.head()

In [ ]:
all_pheno_pc=case_control_gia.merge(all_eigenvec_sub, on = 'person_id')
all_pheno_pc.drop(columns = ['ancestry_pred'], inplace = True)
print(len(all_pheno_pc.index))
print(all_pheno_pc['AD'].value_counts())
print(all_pheno_pc[['AD', 'sex_at_birth']].value_counts())
all_pheno_pc.head()

# make new sample lists

In [ ]:
eur_pheno_pc_sample = eur_pheno_pc[['person_id']]
print(len(eur_pheno_pc_sample.index))
eur_pheno_pc_sample.head()

In [ ]:
all_pheno_pc_sample = all_pheno_pc[['person_id']]
print(len(all_pheno_pc_sample.index))
all_pheno_pc_sample.head()

# export

In [ ]:
eur_pheno_pc.to_csv('AOU.AD.EUR.phenotype_covariates.txt', sep = '\t', index = None)

In [ ]:
all_pheno_pc.to_csv('AOU.AD.ALL.phenotype_covariates.txt', sep = '\t', index = None)

In [ ]:
eur_pheno_pc_sample.to_csv('AOU.AD.EUR.postPCA_QC.sample_list.txt', sep = '\t', index = None, header = None)

In [ ]:
all_pheno_pc_sample.to_csv('AOU.AD.ALL.postPCA_QC.sample_list.txt', sep = '\t', index = None, header = None)